## Cleaning the Data
-----

### Setup:
---

In [1]:
# Import dependencies
import pandas as pd
import datetime as dt

In [2]:
# Grab the CSV
athelete_stats_df = pd.read_csv("initial_unclean_csv/US_unclean_data.csv", encoding="utf8")

In [3]:
# Display as dataframe
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height,weight,birthdate,nationality
0,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,8,3,0,0,6,https://a.espncdn.com/i/headshots/soccer/playe...,"6' 4""",196 lbs,2000-11-07T08:00Z,Ghana
1,David Bingham,Portland Timbers,MLS,Goalkeeper,1.0,0,0,2,0,0,0,49,6,29,https://a.espncdn.com/i/headshots/soccer/playe...,"6' 2""",183 lbs,1989-10-19T07:00Z,USA
2,Ashley Westwood,Charlotte FC,MLS,Midfielder,8.0,3,4,6,0,18,3,6,6,45,https://a.espncdn.com/i/headshots/soccer/playe...,"5' 9""",174 lbs,1990-04-01T08:00Z,England
3,Sean Davis,Nashville SC,MLS,Midfielder,54.0,0,1,0,0,11,1,18,8,21,https://a.espncdn.com/i/headshots/soccer/playe...,"6' 0""",163 lbs,1993-02-08T08:00Z,USA
4,Steve Clark,Houston Dynamo FC,MLS,Goalkeeper,12.0,0,2,5,0,0,0,90,13,40,https://a.espncdn.com/i/headshots/soccer/playe...,"6' 2""",190 lbs,1986-04-14T08:00Z,USA


### Cleaning:
---

#### Convert Height

In [4]:
# Split current heights
split_heights = athelete_stats_df["height"]
split_heights = split_heights.str.split("'", n=1, expand=True)

In [5]:
# Remove everthing but the numbers from the feet
# Convert to float for math
heights_feet = split_heights[0].astype("float")
heights_feet.head(5)

0    6.0
1    6.0
2    5.0
3    6.0
4    6.0
Name: 0, dtype: float64

In [6]:
# Remove everything but the nubmers from the inches
heights_inches = split_heights[1]

split_heights_inches = heights_inches.str.split('"', n=1, expand=True)

# Convert to float for math
split_heights_inches = split_heights_inches[0].astype("float")
heights_inches = split_heights_inches
heights_inches.head(5)

0    4.0
1    2.0
2    9.0
3    0.0
4    2.0
Name: 0, dtype: float64

In [7]:
# Converting inches to feet
inches_in_feet = []
for measurement in heights_inches:
    cur_feet = measurement/12
    inches_in_feet.append(cur_feet)
   

In [8]:
# Create dataframe with new heights colums, both in feet
split_heights_in_ft = {
    "heights_ft1": heights_feet,
    "heights_ft2": inches_in_feet
}
split_heights_in_ft_df = pd.DataFrame(split_heights_in_ft)
split_heights_in_ft_df.head(5)

,heights_ft1,heights_ft2
0,6.0,0.333333
1,6.0,0.166667
2,5.0,0.750000
3,6.0,0.000000
4,6.0,0.166667


In [9]:
# Adding feet and converted inches column to get new total feet measurments column with decimals
new_height = split_heights_in_ft_df["heights_ft1"] + split_heights_in_ft_df["heights_ft2"]

# Round the combined height
new_height = round(new_height,2)

In [10]:
# Display cleaned data
new_height.head(5)

0    6.33
1    6.17
2    5.75
3    6.00
4    6.17
dtype: float64

#### Convert Weight

In [11]:
# Grab uncleaned weight
uncleaned_weight = athelete_stats_df["weight"]
uncleaned_weight.head(5)

0    196 lbs
1    183 lbs
2    174 lbs
3    163 lbs
4    190 lbs
Name: weight, dtype: object

In [12]:
# Split weights
split_weights = uncleaned_weight.str.split(' ', n=1, expand=True)
split_weights.head(5)

,0,1
0,196,lbs
1,183,lbs
2,174,lbs
3,163,lbs
4,190,lbs


In [13]:
# Display cleaned data
new_weight = split_weights[0]
new_weight.head(5)

0    196
1    183
2    174
3    163
4    190
Name: 0, dtype: object

#### Convert Dates

In [14]:
# Split Current Dates
split_birthdates = athelete_stats_df["birthdate"]
split_birthdates = split_birthdates.str.split("T", n=1, expand=True)

In [15]:
# Select only the part we want to keep
split_birthdates = split_birthdates [0]
split_birthdates.head(5)

0    2000-11-07
1    1989-10-19
2    1990-04-01
3    1993-02-08
4    1986-04-14
Name: 0, dtype: object

In [16]:
# Convert the dates to datetime
split_birthdates = pd.to_datetime(split_birthdates)
split_birthdates.head(5)

0   2000-11-07
1   1989-10-19
2   1990-04-01
3   1993-02-08
4   1986-04-14
Name: 0, dtype: datetime64[ns]

In [17]:
# Check dates converted to timestamps
type (split_birthdates[0])

pandas._libs.tslibs.timestamps.Timestamp

In [18]:
# Display cleaned data
new_birthdates = split_birthdates
new_birthdates.head(5)

0   2000-11-07
1   1989-10-19
2   1990-04-01
3   1993-02-08
4   1986-04-14
Name: 0, dtype: datetime64[ns]

#### Remove "Goals Against" and "Clean Sheets" for Non-Goalies

In [19]:
# Remove goals against and clean sheets for non-goalies
new_goals_against = []
new_clean_sheets = []

for index, row in athelete_stats_df.iterrows():
    cur_position = row["position"]
    cur_goals_against = row["goals_against"]
    cur_clean_sheets = row["clean_sheets"]
    
    if cur_position == "Goalkeeper":
        new_goals_against.append(cur_goals_against)
        new_clean_sheets.append(cur_clean_sheets)
    else:
        new_goals_against.append(0)
        new_clean_sheets.append(0)


#### Creating Cleaned Dataframe

In [20]:
# Remove unclean sections from dataframe
athelete_stats_df = athelete_stats_df.drop(["birthdate"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["height"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["weight"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["goals_against"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["clean_sheets"], axis=1)
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality
0,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,8,3,0,https://a.espncdn.com/i/headshots/soccer/playe...,Ghana
1,David Bingham,Portland Timbers,MLS,Goalkeeper,1.0,0,0,2,0,0,0,49,https://a.espncdn.com/i/headshots/soccer/playe...,USA
2,Ashley Westwood,Charlotte FC,MLS,Midfielder,8.0,3,4,6,0,18,3,6,https://a.espncdn.com/i/headshots/soccer/playe...,England
3,Sean Davis,Nashville SC,MLS,Midfielder,54.0,0,1,0,0,11,1,18,https://a.espncdn.com/i/headshots/soccer/playe...,USA
4,Steve Clark,Houston Dynamo FC,MLS,Goalkeeper,12.0,0,2,5,0,0,0,90,https://a.espncdn.com/i/headshots/soccer/playe...,USA


In [21]:
# Add clean sections to dataframe
athelete_stats_df["birthdate"] = new_birthdates
athelete_stats_df["height_(ft)"] = new_height
athelete_stats_df["weight_(lbs)"] = new_weight
athelete_stats_df["goals_against"] = new_goals_against
athelete_stats_df["clean_sheets"] = new_clean_sheets
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality,birthdate,height_(ft),weight_(lbs),goals_against,clean_sheets
0,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,8,3,0,https://a.espncdn.com/i/headshots/soccer/playe...,Ghana,2000-11-07,6.33,196,0,0
1,David Bingham,Portland Timbers,MLS,Goalkeeper,1.0,0,0,2,0,0,0,49,https://a.espncdn.com/i/headshots/soccer/playe...,USA,1989-10-19,6.17,183,29,6
2,Ashley Westwood,Charlotte FC,MLS,Midfielder,8.0,3,4,6,0,18,3,6,https://a.espncdn.com/i/headshots/soccer/playe...,England,1990-04-01,5.75,174,0,0
3,Sean Davis,Nashville SC,MLS,Midfielder,54.0,0,1,0,0,11,1,18,https://a.espncdn.com/i/headshots/soccer/playe...,USA,1993-02-08,6.00,163,0,0
4,Steve Clark,Houston Dynamo FC,MLS,Goalkeeper,12.0,0,2,5,0,0,0,90,https://a.espncdn.com/i/headshots/soccer/playe...,USA,1986-04-14,6.17,190,40,13


In [22]:
# Reorder dataframe
athelete_stats_df = athelete_stats_df[["player_name",
                                       "team",
                                       "league",
                                       "position",
                                       "jersey_number",
                                       "goals",
                                       "assists",
                                       "yellow_cards",
                                       "red_cards",
                                       "shots",
                                       "shots_on_goal",
                                       "saves",
                                       "clean_sheets",
                                       "goals_against",
                                       "photo_url",
                                       "height_(ft)",
                                       "weight_(lbs)",
                                       "birthdate",
                                       "nationality"
                                       ]]
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality
0,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,8,3,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.33,196,2000-11-07,Ghana
1,David Bingham,Portland Timbers,MLS,Goalkeeper,1.0,0,0,2,0,0,0,49,6,29,https://a.espncdn.com/i/headshots/soccer/playe...,6.17,183,1989-10-19,USA
2,Ashley Westwood,Charlotte FC,MLS,Midfielder,8.0,3,4,6,0,18,3,6,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.75,174,1990-04-01,England
3,Sean Davis,Nashville SC,MLS,Midfielder,54.0,0,1,0,0,11,1,18,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.00,163,1993-02-08,USA
4,Steve Clark,Houston Dynamo FC,MLS,Goalkeeper,12.0,0,2,5,0,0,0,90,13,40,https://a.espncdn.com/i/headshots/soccer/playe...,6.17,190,1986-04-14,USA


### Creating Dataframes for SQL Tables:
---

#### Creating Leagues Dataframe

In [23]:
leagues = athelete_stats_df["league"].unique()
leagues_df = pd.DataFrame(leagues)
leagues_df = leagues_df.reset_index()
leagues_df = leagues_df.rename(columns={"index":"league_id", 0:"league"})
leagues_df["league_id"] = leagues_df["league_id"]+1
leagues_df["league_id"] = ["lg" + str(league_id) for league_id in leagues_df["league_id"]]
leagues_df

,league_id,league
0,lg1,MLS
1,lg2,NWSL


#### Creating Teams Dataframe

In [24]:
teams = athelete_stats_df["team"].unique()
teams_df = pd.DataFrame(teams)
teams_df = teams_df.reset_index()
teams_df = teams_df.rename(columns={"index":"team_id", 0:"team"})
teams_df["team_id"] = teams_df["team_id"]+1
teams_df["team_id"] = ["tm" + str(team_id) for team_id in teams_df["team_id"]]
teams_df.head(5)

,team_id,team
0,tm1,Charlotte FC
1,tm2,Portland Timbers
2,tm3,Nashville SC
3,tm4,Houston Dynamo FC
4,tm5,Inter Miami CF


In [25]:
##UNFINISHED##
# Section completed later

#### Creating Positions Dataframe

In [26]:
positions = athelete_stats_df["position"].unique()
positions_df = pd.DataFrame(positions)
positions_df = positions_df.reset_index()
positions_df = positions_df.rename(columns={"index":"position_id", 0:"position"})
positions_df["position_id"] = positions_df["position_id"]+1
positions_df["position_id"] = ["pos" + str(position_id) for position_id in positions_df["position_id"]]
positions_df

,position_id,position
0,pos1,Forward
1,pos2,Goalkeeper
2,pos3,Midfielder
3,pos4,Defender


#### Creating Nationalities Dataframe

In [27]:
nationalities = athelete_stats_df["nationality"].unique()
nationalities_df = pd.DataFrame(nationalities)
nationalities_df = nationalities_df.reset_index()
nationalities_df = nationalities_df.rename(columns={"index":"nationality_id", 0:"nationality"})
nationalities_df["nationality_id"] = nationalities_df["nationality_id"]+1
nationalities_df["nationality_id"] = ["nat" + str(nationality_id) for nationality_id in nationalities_df["nationality_id"]]
nationalities_df.head(5)

,nationality_id,nationality
0,nat1,Ghana
1,nat2,USA
2,nat3,England
3,nat4,Costa Rica
4,nat5,Mexico


#### Creating Player Data Dataframe

In [28]:
# Add player ID
player_data_df = athelete_stats_df
player_data_df = player_data_df.reset_index()
player_data_df = player_data_df.rename(columns={"index":"player_id"})
player_data_df["player_id"] = player_data_df["player_id"]+1
player_data_df["player_id"] = ["pl" + str(player_id) for player_id in player_data_df["player_id"]]
player_data_df.head(5)

,player_id,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality
0,pl1,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,8,3,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.33,196,2000-11-07,Ghana
1,pl2,David Bingham,Portland Timbers,MLS,Goalkeeper,1.0,0,0,2,0,0,0,49,6,29,https://a.espncdn.com/i/headshots/soccer/playe...,6.17,183,1989-10-19,USA
2,pl3,Ashley Westwood,Charlotte FC,MLS,Midfielder,8.0,3,4,6,0,18,3,6,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.75,174,1990-04-01,England
3,pl4,Sean Davis,Nashville SC,MLS,Midfielder,54.0,0,1,0,0,11,1,18,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.00,163,1993-02-08,USA
4,pl5,Steve Clark,Houston Dynamo FC,MLS,Goalkeeper,12.0,0,2,5,0,0,0,90,13,40,https://a.espncdn.com/i/headshots/soccer/playe...,6.17,190,1986-04-14,USA


In [29]:
# Add ID columns
player_data_df = pd.merge(player_data_df,teams_df, on=["team"], how="inner")
player_data_df = pd.merge(player_data_df,leagues_df, on=["league"], how="inner")
player_data_df = pd.merge(player_data_df,positions_df, on=["position"], how="inner")
player_data_df = pd.merge(player_data_df,nationalities_df, on=["nationality"], how="inner")
player_data_df

,player_id,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,...,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality,team_id,league_id,position_id,nationality_id
0,pl1,Patrick Agyemang,Charlotte FC,MLS,Forward,33.0,2,0,3,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.33,196,2000-11-07,Ghana,tm1,lg1,pos1,nat1
1,pl697,Kwadwo Opoku,CF Montréal,MLS,Forward,90.0,6,3,1,1,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.58,152,2001-07-13,Ghana,tm17,lg1,pos1,nat1
2,pl825,Joshua Bolma,New England Revolution,MLS,Forward,16.0,0,0,0,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.75,132,2002-04-10,Ghana,tm24,lg1,pos1,nat1
3,pl223,Eugene Ansah,FC Dallas,MLS,Forward,31.0,1,0,0,0,...,0,NaN,NaN,NaN,1994-12-16,Ghana,tm27,lg1,pos1,nat1
4,pl776,Shakur Mohammed,Orlando City SC,MLS,Midfielder,14.0,0,0,0,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.83,159,2003-08-27,Ghana,tm6,lg1,pos3,nat1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,pl358,Aljaz Ivacic,Portland Timbers,MLS,Goalkeeper,31.0,0,0,0,0,...,26,https://a.espncdn.com/i/headshots/soccer/playe...,6.17,181,1993-12-29,Slovenia,tm2,lg1,pos2,nat64
1170,pl745,Aimé Mabika,Toronto FC,MLS,Defender,6.0,0,0,2,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.50,214,1998-08-16,Zambia,tm9,lg1,pos4,nat82
1171,pl83,Steven Beitashour,Colorado Rapids,MLS,Defender,33.0,0,0,0,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.83,172,1987-02-01,Iran,tm14,lg1,pos4,nat30
1172,pl201,Carlos Akapo,San Jose Earthquakes,MLS,Defender,29.0,2,4,3,0,...,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.83,152,1993-03-12,Equatorial Guinea,tm18,lg1,pos4,nat49


In [30]:
# Drop sections no longer needed
player_data_df = player_data_df.drop(["team"], axis=1)
player_data_df = player_data_df.drop(["league"], axis=1)
player_data_df = player_data_df.drop(["position"], axis=1)
player_data_df = player_data_df.drop(["nationality"], axis=1)

In [31]:
# Reorder dataframe
player_data_df = player_data_df[[
                                "player_id",
                                "player_name",
                                "team_id",
                                "league_id",
                                "position_id",
                                "jersey_number",
                                "goals",
                                "assists",
                                "yellow_cards",
                                "red_cards",
                                "shots",
                                "shots_on_goal",
                                "saves",
                                "clean_sheets",
                                "goals_against",
                                "photo_url",
                                "height_(ft)",
                                "weight_(lbs)",
                                "birthdate",
                                "nationality_id"
                            ]]
player_data_df.head()

,player_id,player_name,team_id,league_id,position_id,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality_id
0,pl1,Patrick Agyemang,tm1,lg1,pos1,33.0,2,0,3,0,8,3,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.33,196,2000-11-07,nat1
1,pl697,Kwadwo Opoku,tm17,lg1,pos1,90.0,6,3,1,1,51,18,2,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.58,152,2001-07-13,nat1
2,pl825,Joshua Bolma,tm24,lg1,pos1,16.0,0,0,0,0,1,0,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.75,132,2002-04-10,nat1
3,pl223,Eugene Ansah,tm27,lg1,pos1,31.0,1,0,0,0,8,4,0,0,0,NaN,NaN,NaN,1994-12-16,nat1
4,pl776,Shakur Mohammed,tm6,lg1,pos3,14.0,0,0,0,0,0,0,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.83,159,2003-08-27,nat1


In [32]:
# Complete teams dataframe
teams_df = pd.merge(teams_df,player_data_df[["team_id","league_id"]], on=["team_id"], how ="left")
teams_df = teams_df.drop_duplicates()
teams_df = teams_df.reset_index(drop=True)
teams_df

,team_id,team,league_id
0,tm1,Charlotte FC,lg1
1,tm2,Portland Timbers,lg1
2,tm3,Nashville SC,lg1
3,tm4,Houston Dynamo FC,lg1
4,tm5,Inter Miami CF,lg1
5,tm6,Orlando City SC,lg1
6,tm7,LAFC,lg1
7,tm8,Austin FC,lg1
8,tm9,Toronto FC,lg1
9,tm10,New York City FC,lg1


#### Correct Data Types

In [33]:
# Check data types
player_data_df.dtypes

player_id                 object
player_name               object
team_id                   object
league_id                 object
position_id               object
jersey_number            float64
goals                      int64
assists                    int64
yellow_cards               int64
red_cards                  int64
shots                      int64
shots_on_goal              int64
saves                      int64
clean_sheets               int64
goals_against              int64
photo_url                 object
height_(ft)              float64
weight_(lbs)              object
birthdate         datetime64[ns]
nationality_id            object
dtype: object

In [34]:
# Set data type for jersey numbers
jersey_int=[]
for jersey_number in player_data_df["jersey_number"]:
    
    try:
        new_type = int(jersey_number)
    except:
       new_type = -1
    
    jersey_int.append(new_type)
    
print(jersey_int)

[33, 90, 16, 31, 14, 11, 18, 14, 30, 6, 15, 4, 22, 11, 11, 11, 7, 32, 19, 43, 9, 10, 10, 22, 22, 7, 37, 17, 11, 7, 5, 9, 20, 7, 24, 11, 11, 7, 10, 15, 27, 7, 10, 5, 10, 29, 10, 31, 7, 22, 19, 5, 10, 31, 5, 2, 33, 6, 15, 3, 3, 21, 80, 43, 10, 9, 9, 19, 11, 17, 10, 9, 19, 9, 20, 6, 7, 26, 8, 21, 20, 51, 8, 12, 6, 23, 93, 10, 82, 77, 8, 99, 6, 5, 31, 25, 3, 13, 26, 2, 2, 78, 4, 30, 3, 42, 18, 27, 30, 21, 28, 12, 12, 9, 12, 9, 8, 21, 9, 21, 23, 77, 5, 21, 13, 11, 21, 11, 50, 29, 3, 28, 4, 4, 4, 5, 94, 4, 13, 15, 7, 9, 11, 13, 17, 16, 12, 11, 19, 45, 27, 13, 95, 38, 29, 9, 9, 22, 36, 36, 28, 11, 31, 7, 77, 11, 16, 46, 13, 35, 13, 44, 28, 12, 20, 8, 33, 32, 17, 23, 21, 11, 9, 29, 24, 17, 12, 47, 25, 17, 38, 37, 27, 7, 10, 27, 11, 77, 74, 17, 41, 22, 19, 9, 33, 24, 45, 21, 39, 5, 14, 17, 22, 77, 25, 5, 17, 13, 15, 3, 7, 11, 25, 28, 23, 15, 24, 23, 34, 8, 10, 23, 30, -1, 33, 20, 9, 10, 24, 2, 2, 7, 3, 9, 8, 24, 15, 55, 25, 34, 11, 5, 20, 14, 13, 35, 15, 34, 58, 19, 54, 6, 19, 26, 11, 25, 35, 3

In [35]:
# Set data type for weight
weight_int=[]
for weight in player_data_df["weight_(lbs)"]:
    
    try:
        new_type = int(weight)
    except:
       new_type = -1
    
    weight_int.append(new_type)
    
print(weight_int)

[196, 152, 132, -1, 159, 139, 150, 146, -1, 181, 161, 183, 168, 170, 170, 192, 152, 117, 150, 183, 161, 150, 159, 159, 179, 150, 163, 159, 161, 159, 168, 146, 139, 119, 137, 152, 128, 192, 174, 150, 115, 150, 150, 172, 139, 139, 161, 150, 174, 161, 148, 161, 132, 146, 150, 159, 152, 159, 181, 181, 157, 183, -1, 152, 152, 183, 183, 170, 163, 183, 128, 117, -1, 117, 168, 150, 159, 172, 152, 159, 163, 172, 161, 161, 157, 150, 168, 157, 130, 130, 130, 117, 128, 130, 181, 172, 161, 172, 168, 141, -1, 141, 150, -1, 119, 203, 152, 163, 148, -1, 152, 132, 161, 152, 152, 183, 137, 163, 168, 150, 170, 146, 148, 150, 174, 132, 146, 146, -1, 159, 159, 203, 161, 185, 170, 192, 163, 179, 159, 168, 170, 150, 170, 152, 196, 146, 174, 163, 170, -1, 172, -1, -1, 172, 192, 174, 183, 150, 152, 172, 152, 161, 207, 150, 172, -1, 201, -1, 183, 161, 181, 172, 163, 174, 170, 163, 172, 161, 170, 179, 161, 190, 172, 185, 183, 172, 163, 139, 161, 185, 168, 150, 172, 146, 150, 150, 174, 150, 183, 157, -1, 163, 196

In [36]:
# Replace converted sections
player_data_df["jersey_number"] = jersey_int
player_data_df["weight_(lbs)"] = weight_int

In [37]:
# Drop league ID
player_data_df = player_data_df.drop(["league_id"], axis=1)

In [38]:
# Deal with na values
player_data_df["goals"] = player_data_df["goals"].fillna(0)
player_data_df["assists"] = player_data_df["assists"].fillna(0)
player_data_df["yellow_cards"] = player_data_df["yellow_cards"].fillna(0)
player_data_df["red_cards"] = player_data_df["red_cards"].fillna(0)
player_data_df["shots"] = player_data_df["shots"].fillna(0)
player_data_df["shots_on_goal"] = player_data_df["shots_on_goal"].fillna(0)
player_data_df["saves"] = player_data_df["saves"].fillna(0)
player_data_df["clean_sheets"] = player_data_df["clean_sheets"].fillna(0)
player_data_df["goals_against"] = player_data_df["goals_against"].fillna(0)
player_data_df["photo_url"] = player_data_df["photo_url"].fillna("N/A")
player_data_df["height_(ft)"] = player_data_df["height_(ft)"].fillna(0)
player_data_df["weight_(lbs)"] = player_data_df["weight_(lbs)"].fillna(0)
player_data_df["birthdate"] = player_data_df["birthdate"].fillna(dt.date(1111,11,11))

In [39]:
# Display Player Data dataframe
player_data_df.head(10)

,player_id,player_name,team_id,position_id,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality_id
0,pl1,Patrick Agyemang,tm1,pos1,33,2,0,3,0,8,3,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.33,196,2000-11-07 00:00:00,nat1
1,pl697,Kwadwo Opoku,tm17,pos1,90,6,3,1,1,51,18,2,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.58,152,2001-07-13 00:00:00,nat1
2,pl825,Joshua Bolma,tm24,pos1,16,0,0,0,0,1,0,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.75,132,2002-04-10 00:00:00,nat1
3,pl223,Eugene Ansah,tm27,pos1,31,1,0,0,0,8,4,0,0,0,N/A,0.00,-1,1994-12-16 00:00:00,nat1
4,pl776,Shakur Mohammed,tm6,pos3,14,0,0,0,0,0,0,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.83,159,2003-08-27 00:00:00,nat1
5,pl412,Latif Blessing,tm9,pos3,11,0,1,4,1,6,2,3,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.58,139,1996-12-30 00:00:00,nat1
6,pl207,Emmanuel Boateng,tm24,pos3,18,2,3,1,0,17,6,4,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.50,150,1994-01-17 00:00:00,nat1
7,pl157,Yaw Yeboah,tm26,pos3,14,3,6,3,0,35,13,1,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,5.50,146,1997-03-28 00:00:00,nat1
8,pl794,Samuel Kwaku Owusu,tm10,pos4,30,0,0,0,0,0,0,0,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,0.00,-1,2000-12-13 00:00:00,nat1
9,pl414,Lalas Abubakar,tm14,pos4,6,2,0,8,1,8,5,19,0,0,https://a.espncdn.com/i/headshots/soccer/playe...,6.08,181,1994-12-25 00:00:00,nat1


In [40]:
player_data_df.dtypes

player_id          object
player_name        object
team_id            object
position_id        object
jersey_number       int64
goals               int64
assists             int64
yellow_cards        int64
red_cards           int64
shots               int64
shots_on_goal       int64
saves               int64
clean_sheets        int64
goals_against       int64
photo_url          object
height_(ft)       float64
weight_(lbs)        int64
birthdate          object
nationality_id     object
dtype: object

In [41]:
teams_df.dtypes

team_id      object
team         object
league_id    object
dtype: object

In [42]:
leagues_df.dtypes

league_id    object
league       object
dtype: object

In [43]:
positions_df.dtypes

position_id    object
position       object
dtype: object

In [44]:
nationalities_df.dtypes

nationality_id    object
nationality       object
dtype: object

### Exports:
---

In [45]:
# Export CSV 1-Player Data
player_data_df.to_csv("../2-database_building/cleaned_data_exports/us_player_data.csv", index=False)

In [46]:
# Export CSV 2-Teams Data
teams_df.to_csv("../2-database_building/cleaned_data_exports/us_teams.csv", index=False)

In [47]:
# Export CSV 3-Leagues Data
leagues_df.to_csv("../2-database_building/cleaned_data_exports/us_leagues.csv", index=False)

In [48]:
# Export CSV 4-Positions Data
positions_df.to_csv("../2-database_building/cleaned_data_exports/us_positions.csv", index=False)

In [49]:
# Export CSV 5-Nationalities Data
nationalities_df.to_csv("../2-database_building/cleaned_data_exports/us_nationalities.csv", index=False)